# Network data exploration

## Load data

Gather paths from private json file

In [ ]:
import json

with open('paths.json') as f:
    json_data = json.load(f)
json_data

In [ ]:
networkfile = json_data['data']['file']

Read network data from file

In [ ]:
import scapy.all

a = scapy.all.rdpcap(networkfile)
sessions = a.sessions()

print('Loaded {} sessions'.format(len(sessions)))

In [ ]:
sessionKeys = list(sessions.keys())
print(sessionKeys[0:2])
sessionVals = list(sessions.values())
print(sessionVals[0:2])

## Investigate pcap data

### Investigate single packet list

In [ ]:
sessionVals[0].show()

In [ ]:
sessionVals[0].make_table(lambda x:(x[scapy.all.IP].dport, x[scapy.all.TCP].dport, x[scapy.all.TCP].payload_guess))

In [ ]:
sessionVals[0][scapy.all.TCP][1]

### Fill data in dataframe

Define extractors for a single packet

In [ ]:
import numpy as np

errors = {}

def catch_exception(func):
    def func_wrapper(*args, **kargs):
        try:
            return func(*args, **kargs)    
        except (IndexError, AttributeError) as e:
            err = e.__class__.__name__ + ': ' + str(*e.args)
            if err not in errors.keys():
                errors[err] = 1
            else:
                errors[err] += 1
            return [np.NaN] * len(args[0])
    return func_wrapper

In [ ]:
# define IP functions

@catch_exception
def extract_IP_flags(packet):
    return list(map(lambda x: x[scapy.all.IP].sprintf("%flags%"), packet))

@catch_exception
def extract_IP_time(packet):
    return list(map(lambda x: x[scapy.all.IP].time, packet))

@catch_exception
def extract_destination_IPs(packet):
    return list(map(lambda x: x[scapy.all.IP].dst, packet))

@catch_exception
def extract_destination_IP_ports(packet):
    return list(map(lambda x: x[scapy.all.IP].dport, packet))

@catch_exception
def extract_source_IP_ports(packet):
    return list(map(lambda x: x[scapy.all.IP].sport, packet))


# define UDP functions

@catch_exception
def extract_destination_UDP_ports(packet):
    return list(map(lambda x: x[scapy.all.UDP].dport, packet))


# define TCP functions

@catch_exception
def extract_TCP_flags(packet):
    return list(map(lambda x: x[scapy.all.TCP].sprintf("%flags%"), packet))

@catch_exception
def extract_destination_TCP_ports(packet):
    return list(map(lambda x: x[scapy.all.TCP].dport, packet))

Perform extraction for all packets

In [ ]:
import itertools

def flatten_map(l):
    return list(itertools.chain(*list(l)))

def extract_data(packets):
    print('This packet list has ' + str(len(packets)) + ' packets')
    destination_IPs = flatten_map(map(extract_destination_IPs, packets))
    destination_IP_ports = flatten_map(map(extract_destination_IP_ports, packets))
    destination_TCP_ports = flatten_map(map(extract_destination_TCP_ports, packets))
    TCP_flags = flatten_map(map(extract_TCP_flags, packets))
    IP_flags = flatten_map(map(extract_IP_flags, packets))
    destination_UDP_ports = flatten_map(map(extract_destination_UDP_ports, packets))
    IP_time = flatten_map(map(extract_IP_time, packets))
    source_IP_ports = flatten_map(map(extract_source_IP_ports, packets))
    
    data = {}
    data['IP_time'] = IP_time
    data['source_IPs'] = source_IP_ports
    data['destination_IPs'] = destination_IPs
    data['destination_IP_ports'] = destination_IP_ports
    data['destination_TCP_ports'] = destination_TCP_ports
    data['TCP_flags'] = TCP_flags
    data['IP_flags'] = IP_flags
    data['destination_UDP_ports'] = destination_UDP_ports

    return data

In [ ]:
data = extract_data(sessionVals[0:])
print(errors)

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df

In [ ]:
df.sort_values(by=['IP_time'])

### Plot data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns



In [ ]:
 # Print the location of Jupyter's config directory
 from jupyter_core.paths import jupyter_config_dir
 jupyter_dir = jupyter_config_dir()
 print(jupyter_dir)

 # Print the location of custom.js
 import os.path
 custom_js_path = os.path.join(jupyter_dir, 'custom', 'custom.js')
 print(custom_js_path)

 # Print the contents of custom.js, if it exists.
 if os.path.isfile(custom_js_path):
     with open(custom_js_path) as f:
         print(f.read())
 else:
     print("You don't have a custom.js file")